In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lrex").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/17 21:18:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
pip install numpy

  Using cached numpy-1.22.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
from pyspark.ml.regression import LinearRegression

In [8]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

22/04/17 21:25:53 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [9]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [10]:
lr = LinearRegression(featuresCol = "features", labelCol = "label", predictionCol = "prediction")

In [11]:
lrModel = lr.fit(training)

22/04/17 21:31:11 WARN Instrumentation: [aa96deff] regParam is zero, which might cause numerical instability and overfitting.
22/04/17 21:31:11 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
lrModel.intercept

0.14228558260358093

In [14]:
training_summary = lrModel.summary

In [15]:
training_summary.r2

0.027839179518600154

In [18]:
training_summary.rootMeanSquaredError

10.16309157133015

In [19]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

22/04/17 21:37:51 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [20]:
split_object = all_data.randomSplit([0.7, 0.3])

In [21]:
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [22]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [23]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
+----------

In [25]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                362|
|   mean| 0.4008582380510088|
| stddev| 10.394635164947733|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [26]:
test_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 139|
|   mean|-0.11805310974658852|
| stddev|   10.14276155531714|
|    min| -26.736207182601724|
|    max|  24.290551295953957|
+-------+--------------------+



In [27]:
 correct_model = lr.fit(train_data)

22/04/17 21:45:23 WARN Instrumentation: [8c63f146] regParam is zero, which might cause numerical instability and overfitting.


In [28]:
test_results = correct_model.evaluate(test_data)

In [29]:
test_results.residuals.show()

/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------------+
|          residuals|
+-------------------+
|-22.320263781411736|
| -23.59927213850935|
|-28.332855890504824|
|-20.946709662793406|
| -20.60782658658802|
|-17.765639702929672|
|  -14.2079526356371|
|-19.528866314550307|
|-18.485171117635527|
| -13.74658350775795|
|-21.143214806750848|
|-11.434498363298388|
|-11.839078921682294|
|-17.457169291620545|
|   -14.856896728324|
| -17.67431120536545|
| -15.51264393428923|
|-13.583804065872775|
| -9.050921255095659|
|-12.061500211654309|
+-------------------+
only showing top 20 rows



In [30]:
test_results.rootMeanSquaredError

10.43147582069004

In [31]:
unlabeled_data = test_data.select("features")

In [32]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [33]:
predictions = correct_model.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -4.415943401189987|
|(10,[0,1,2,3,4,5,...|0.11183201757283823|
|(10,[0,1,2,3,4,5,...|  5.383029954308748|
|(10,[0,1,2,3,4,5,...| 0.8892270470041952|
|(10,[0,1,2,3,4,5,...|  0.723265812314594|
|(10,[0,1,2,3,4,5,...|-1.6366963272848818|
|(10,[0,1,2,3,4,5,...| -3.286247721246242|
|(10,[0,1,2,3,4,5,...| 2.4634666886742895|
|(10,[0,1,2,3,4,5,...| 1.7929640963244229|
|(10,[0,1,2,3,4,5,...| -2.204929058036623|
|(10,[0,1,2,3,4,5,...|  5.362529774127545|
|(10,[0,1,2,3,4,5,...| -3.914372792080866|
|(10,[0,1,2,3,4,5,...|-3.4719016677339947|
|(10,[0,1,2,3,4,5,...|  3.128190782545103|
|(10,[0,1,2,3,4,5,...| 0.8807657971712954|
|(10,[0,1,2,3,4,5,...|  4.752088101995029|
|(10,[0,1,2,3,4,5,...| 3.3145473696278165|
|(10,[0,1,2,3,4,5,...|  1.489452787337516|
|(10,[0,1,2,3,4,5,...| -2.806429110333766|
|(10,[0,1,2,3,4,5,...| 0.4457249466386819|
+----------